Modelo de Regresión Logistica en Python
=======================================

En el articulo anterior, creamos un modelo de predicción de enfermedades cardiacas usando un modelo de regresión logistica en RapidMiner. En este articulo, vamos a crear el mismo modelo usando Python.

## Carga de datos

Utilizaremos el mismo archivo de datos que en el articulo anterior. Este archivo fue creado luego de procesar las cuatro bases de datos originales, conviertiendolas a un formato csv mas fácil de importar a las distintas herramientas de aprendizaje automatizado

In [856]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [857]:
dataset: pd.DataFrame = pd.read_csv('./datasets/procesados/full.csv')

Aseguremonos antes de continuar, de que cada columna tenga el tipo de datos correcto.

In [858]:
columns = [
    ('id', 'Int64'),
    ('ccf', 'Int64'),
    ('age', 'Int64'),
    ('sex', 'category'),
    ('painloc', 'category'),
    ('painexer', 'category'),
    ('relrest', 'category'),
    ('pncaden', 'category'),
    ('cp', 'category'),
    ('trestbps', 'float'),
    ('htn', 'object'),
    ('chol', 'float'),
    ('smoke', 'category'),
    ('cigs', 'Int64'),
    ('years', 'Int64'),
    ('fbs', 'category'),
    ('dm', 'category'),
    ('famhist', 'category'),
    ('restecg', 'category'),
    ('ekgmo', 'Int64'),
    ('ekgday', 'Int64'),
    ('ekgyr', 'Int64'),
    ('dig', 'category'),
    ('prop', 'category'),
    ('nitr', 'category'),
    ('pro', 'category'),
    ('diuretic', 'category'),
    ('proto', 'category'),
    ('thaldur', 'float'),
    ('thaltime', 'float'),
    ('met', 'float'),
    ('thalach', 'float'),
    ('thalrest', 'float'),
    ('tpeakbps', 'float'),
    ('tpeakbpd', 'float'),
    ('dummy', 'object'),
    ('trestbpd', 'float'),
    ('exang', 'category'),
    ('xhypo', 'category'),
    ('oldpeak', 'float'),
    ('slope', 'category'),
    ('rldv5', 'float'),
    ('rldv5e', 'float'),
    ('ca', 'Int64'),
    ('restckm', 'object'),
    ('exerckm', 'object'),
    ('restef', 'float'),
    ('restwm', 'category'),
    ('exeref', 'float'),
    ('exerwm', 'float'),
    ('thal', 'category'),
    ('thalsev', 'object'),
    ('thalpul', 'object'),
    ('earlobe', 'object'),
    ('cmo', 'Int64'),
    ('cday', 'Int64'),
    ('cyr', 'Int64'),
    ('num', 'category'),
    ('lmt', 'object'),
    ('ladprox', 'object'),
    ('laddist', 'object'),
    ('diag', 'object'),
    ('cxmain', 'object'),
    ('ramus', 'object'),
    ('om1', 'object'),
    ('om2', 'object'),
    ('rcaprox', 'object'),
    ('rcadist', 'object'),
    ('lvx1', 'object'),
    ('lvx2', 'object'),
    ('lvx3', 'object'),
    ('lvx4', 'object'),
    ('lvf', 'object'),
    ('cathef', 'object'),
    ('junk', 'object'),
    ('name', 'string'),
    ('dataset', 'string')
]

dataset = dataset.astype(dict(columns))

print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 77 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dataset   899 non-null    string  
 1   id        899 non-null    Int64   
 2   ccf       899 non-null    Int64   
 3   age       899 non-null    Int64   
 4   sex       899 non-null    category
 5   painloc   617 non-null    category
 6   painexer  617 non-null    category
 7   relrest   613 non-null    category
 8   pncaden   0 non-null      category
 9   cp        899 non-null    category
 10  trestbps  840 non-null    float64 
 11  htn       865 non-null    object  
 12  chol      869 non-null    float64 
 13  smoke     230 non-null    category
 14  cigs      479 non-null    Int64   
 15  years     467 non-null    Int64   
 16  fbs       809 non-null    category
 17  dm        95 non-null     category
 18  famhist   477 non-null    category
 19  restecg   897 non-null    category
 20  ekgmo     

## Preprocesamiento

En el articulo anterior, realizamos un preprocesamiento de los datos usando RapidMiner. En este caso, realizaremos el preprocesamiento usando Python. Para ello, utilizaremos las librerias Pandas y Numpy.

Inicialmente, tomaremos la variable objetivo `num` y convertiremos todo valor distinto de cero a 1. Esto es necesario para que el modelo de regresión logistica pueda funcionar correctamente ya que es un modelo de clasificación binaria.

In [859]:
for value in dataset['num'].unique():
    if value > 0:
        dataset['num'] = dataset['num'].replace(value, 1)

print(dataset['num'].unique())

[0, 1]
Categories (2, int64): [0, 1]


Luego, tomemos todas las columnas que contienen una cantidad de valores faltantes mayor a 400 y eliminemos esas columnas del dataset. Este numero fue seleccionado basados en que existen 899 registros en el dataset.

In [860]:
dataset = dataset.dropna(axis=1, thresh=400)
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dataset   899 non-null    string  
 1   id        899 non-null    Int64   
 2   ccf       899 non-null    Int64   
 3   age       899 non-null    Int64   
 4   sex       899 non-null    category
 5   painloc   617 non-null    category
 6   painexer  617 non-null    category
 7   relrest   613 non-null    category
 8   cp        899 non-null    category
 9   trestbps  840 non-null    float64 
 10  htn       865 non-null    object  
 11  chol      869 non-null    float64 
 12  cigs      479 non-null    Int64   
 13  years     467 non-null    Int64   
 14  fbs       809 non-null    category
 15  famhist   477 non-null    category
 16  restecg   897 non-null    category
 17  ekgmo     846 non-null    Int64   
 18  ekgday    845 non-null    Int64   
 19  ekgyr     846 non-null    Int64   
 20  dig       

Ahora procedemos a eliminar las columnas que son marcadas por el autor del dataset como no relevantes para el modelo. Estas columnas son `ccf`, `dummy`, `lvx1`, `lvx2`, `lvx3`, `lvx4`, `lvf`, `junk`, y `name`.

In [861]:
columns = [
    'ccf',
    'dummy',
    'lvx1',
    'lvx2',
    'lvx3',
    'lvx4',
    'lvf',
    'name'
]

dataset = dataset.drop(columns, axis=1)
# dataset = dataset.select_dtypes(exclude=['object'])
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899 entries, 0 to 898
Data columns (total 51 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dataset   899 non-null    string  
 1   id        899 non-null    Int64   
 2   age       899 non-null    Int64   
 3   sex       899 non-null    category
 4   painloc   617 non-null    category
 5   painexer  617 non-null    category
 6   relrest   613 non-null    category
 7   cp        899 non-null    category
 8   trestbps  840 non-null    float64 
 9   htn       865 non-null    object  
 10  chol      869 non-null    float64 
 11  cigs      479 non-null    Int64   
 12  years     467 non-null    Int64   
 13  fbs       809 non-null    category
 14  famhist   477 non-null    category
 15  restecg   897 non-null    category
 16  ekgmo     846 non-null    Int64   
 17  ekgday    845 non-null    Int64   
 18  ekgyr     846 non-null    Int64   
 19  dig       831 non-null    category
 20  prop      

En este punto nos encontramos con 51 atributos. Dejemos de lado la seleccion de atributos por el momento y continuemos con otras tareas de preprocesamiento.

Ahora, tomemos las columnas que contienen valores faltantes y reemplacemos esos valores por el promedio de la columna. En el caso de numeros reales, tomemos el promedio de los valores. En el caso de valores discretos, tomemos el valor mas frecuente.

In [862]:
# Reeplazemos valores faltantes de columnas 'float', por la media.
for column in dataset.columns:
    if dataset[column].dtype == 'float64':
        dataset[column] = dataset[column].fillna(dataset[column].mean())

# Reemplazamos valores faltantes de columnas 'category', por la moda.
for column in dataset.columns:
    if dataset[column].dtype == 'category':
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])

# Reemplazamos valores faltantes de columnas 'Int64', por la moda.
for column in dataset.columns:
    if dataset[column].dtype == 'Int64':
        dataset[column] = dataset[column].fillna(dataset[column].mode()[0])

Ya con los valores faltantes que podemos reemplazar, reemplazados, eliminemos las filas que contienen valores faltantes en las columnas restantes.

In [863]:
dataset = dataset.dropna(axis=0)
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 572 entries, 0 to 898
Data columns (total 51 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dataset   572 non-null    string  
 1   id        572 non-null    Int64   
 2   age       572 non-null    Int64   
 3   sex       572 non-null    category
 4   painloc   572 non-null    category
 5   painexer  572 non-null    category
 6   relrest   572 non-null    category
 7   cp        572 non-null    category
 8   trestbps  572 non-null    float64 
 9   htn       572 non-null    object  
 10  chol      572 non-null    float64 
 11  cigs      572 non-null    Int64   
 12  years     572 non-null    Int64   
 13  fbs       572 non-null    category
 14  famhist   572 non-null    category
 15  restecg   572 non-null    category
 16  ekgmo     572 non-null    Int64   
 17  ekgday    572 non-null    Int64   
 18  ekgyr     572 non-null    Int64   
 19  dig       572 non-null    category
 20  prop      572 n

Volvamos a la selección de atributos momentariamente. Vamos a investigar la correlacion entre los atributos, y eliminar aquellos que tengan una correlación mayor al 35%.

In [864]:
correlation_matrix = dataset.drop(['dataset', 'id', 'num'], axis=1).corr().abs()

upper = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

to_drop = [column for column in upper.columns if any(upper[column] > 0.35)]

dataset = dataset.drop(dataset[to_drop], axis=1)
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Index: 572 entries, 0 to 898
Data columns (total 34 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   dataset   572 non-null    string  
 1   id        572 non-null    Int64   
 2   age       572 non-null    Int64   
 3   sex       572 non-null    category
 4   painloc   572 non-null    category
 5   painexer  572 non-null    category
 6   trestbps  572 non-null    float64 
 7   htn       572 non-null    object  
 8   chol      572 non-null    float64 
 9   cigs      572 non-null    Int64   
 10  fbs       572 non-null    category
 11  famhist   572 non-null    category
 12  restecg   572 non-null    category
 13  ekgmo     572 non-null    Int64   
 14  ekgday    572 non-null    Int64   
 15  dig       572 non-null    category
 16  prop      572 non-null    category
 17  nitr      572 non-null    category
 18  pro       572 non-null    category
 19  diuretic  572 non-null    category
 20  thaldur   572 n

In [865]:
# correlation_matrix = dataset.drop(['dataset', 'id', 'num'], axis=1).corr().abs()
# print(correlation_matrix)

# threshold = 0.35

# mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# correlated_pairs = []
# for i in range(correlation_matrix.shape[0]):
#     for j in range(i+1, correlation_matrix.shape[1]):
#         if mask[i, j] and abs(correlation_matrix.iloc[i, j]) > threshold:
#             colname_i = correlation_matrix.columns[i]
#             colname_j = correlation_matrix.columns[j]
#             correlated_pairs.append((colname_i, colname_j))

# to_drop = set()
# for pair in correlated_pairs:
#     to_drop.add(pair[1])

# dataset = dataset.drop(to_drop, axis=1)
# print(dataset.info())

En este punto tenemos un conjunto de datos completo, procedemos a realizar una optimización de atributos usando `forwards selection`, teniendo en consideración el modelo de regresión logística. Realizaremos un `forwards selection` con un numero de rondas especulativas de 10. Para ello, utilizaremos la libreria `sklearn`.

In [866]:
pipeline = Pipeline([
    ('selected_features', SelectFromModel(LogisticRegression(max_iter=1000000), threshold='median')),
    ('scaler', StandardScaler()),
    ('classification', LogisticRegression(max_iter=1000000))
])

Con esta ultima operación de preprocesamiento, nos quedamos con 18 atributos, mas 2 de metadata, y uno de clase. En total, 21 columnas.

## Entrenamiento

Ahora que tenemos el dataset preprocesado, procedemos a entrenar el modelo de regresión logistica. Para ello, continuaremos utilizando la libreria `sklearn`. Este paso es similar al utilizado dentro de la optimización de atributos, pero en este caso, utilizaremos validación cruzada en lugar de una división 70-30.

In [869]:
inputs = dataset.drop(['num'], axis=1).drop(['dataset', 'id'], axis=1)
outputs = dataset['num']

output_predictions = cross_val_predict(pipeline, inputs, outputs, cv=10)
print('ROC AUC Score: \n', roc_auc_score(outputs, output_predictions), end='\n\n')
print('Confusion Matrix: \n', confusion_matrix(outputs, output_predictions), end='\n\n')
print('Classification Report: \n', classification_report(outputs, output_predictions), end='\n\n')

ROC AUC Score: 
 0.9920901655485683

Confusion Matrix: 
 [[210   1]
 [  4 357]]

Classification Report: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       211
           1       1.00      0.99      0.99       361

    accuracy                           0.99       572
   macro avg       0.99      0.99      0.99       572
weighted avg       0.99      0.99      0.99       572




Con una precisión del 99%, podemos decir que el modelo de regresión logistica es un buen modelo para este dataset.